In [1]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
import pyarrow

In [3]:
df = pd.read_parquet("../data/full_dataset_feature_engineering_v2.parquet")

In [ ]:
import pickle
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler


with open('./pickles/hybrid_feature_scaler.pkl', 'rb') as file:
    feature_scaler = pickle.load(file)

with open('./pickles/hybrid_target_scaler.pkl', 'rb') as file:
    target_scaler = pickle.load(file)

with open('./pickles/hybrid_selected_features.pkl', 'rb') as file:
    selected_features = pickle.load(file)

2025-04-21 05:50:33.312786: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-21 05:50:33.317777: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-21 05:50:33.333420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745214633.360181   45066 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745214633.368113   45066 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745214633.388530   45066 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X_df = df[selected_features].values
y_df = df['return_forward'].values  
original_indexes = df.index.tolist()

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_df)


y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y_df.reshape(-1, 1))


sequence_length = 12  


X_sequences = []
y_sequences = []
sequence_indexes = []

for i in range(len(X_scaled) - sequence_length):
    X_sequences.append(X_scaled[i:i+sequence_length])
    y_sequences.append(y_scaled[i+sequence_length])
    sequence_indexes.append(original_indexes[i+sequence_length])


X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)


print(f"X shape: {X_sequences.shape}")  
print(f"y shape: {y_sequences.shape}")  



X_train, X_test, y_train, y_test ,train_idx, test_idx = train_test_split(
    X_sequences, y_sequences, sequence_indexes, test_size=0.25062, shuffle=False
)

print(f"Total sequences: {len(sequence_indexes)}")
print(f"Training sequences: {len(train_idx)}")
print(f"Testing sequences: {len(test_idx)}")


X shape: (35046, 12, 15)
y shape: (35046, 1)
Total sequences: 35046
Training sequences: 26262
Testing sequences: 8784


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class NBeatsGenericBlock(layers.Layer):
    """
    Generic N-BEATS block using fully connected layers.
    """
    def __init__(self,
                 input_timesteps: int,
                 output_timesteps: int,
                 layer_widths: list, 
                 theta_dim: int, 
                 backcast_length_factor: int = 1, 
                 dropout_rate: float = 0.1,
                 activation: str = 'relu',
                 **kwargs):
        super().__init__(**kwargs)
        self.input_timesteps = input_timesteps
        self.output_timesteps = output_timesteps
        self.layer_widths = layer_widths
        self.theta_dim = theta_dim 
        self.backcast_length = input_timesteps * backcast_length_factor
        self.dropout_rate = dropout_rate
        self.activation = activation

        
        self.hidden_layers = []
        for width in layer_widths:
            self.hidden_layers.append(layers.Dense(width, activation=activation))
            if dropout_rate > 0:
                 self.hidden_layers.append(layers.Dropout(dropout_rate))

        
        self.theta_layer = layers.Dense(theta_dim, activation=activation, name='theta')

        
        
        self.forecast_layer = layers.Dense(output_timesteps, activation='linear', name='forecast')
        self.backcast_layer = layers.Dense(self.backcast_length, activation='linear', name='backcast')

    def call(self, inputs, training=False): 
        x = inputs
        for layer in self.hidden_layers:
            
            if isinstance(layer, layers.Dropout):
                x = layer(x, training=training)
            else:
                x = layer(x)

        theta = self.theta_layer(x)

        forecast = self.forecast_layer(theta)
        backcast = self.backcast_layer(theta)

        return forecast, backcast

    
    def get_config(self):
        config = super().get_config()
        config.update({
            'input_timesteps': self.input_timesteps,
            'output_timesteps': self.output_timesteps,
            'layer_widths': self.layer_widths,
            'theta_dim': self.theta_dim,
            'backcast_length_factor': self.backcast_length // self.input_timesteps,
            'dropout_rate': self.dropout_rate,
            'activation': self.activation,
        })
        return config


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def create_nbeats_model(
    input_timesteps: int,
    output_timesteps: int,
    num_stacks: int,         
    theta_dim: int,          
    layer_widths: list,      
    
    num_blocks_per_stack: int = 3, 
    dropout_rate: float = 0.1,
    activation: str = 'relu',
    learning_rate: float = 0.001,
    loss_function: str = 'mean_squared_error'
    ):
    """
    Creates a Generic N-BEATS Keras model.

    Args:
        input_timesteps: Length of the input sequence (lookback window).
        output_timesteps: Length of the output sequence (forecast horizon).
        num_stacks: Number of stacks in the model (Tuned).
        theta_dim: Bottleneck dimension before final projection in blocks (Tuned).
        layer_widths: List of hidden layer widths within each block (Tuned).
        num_blocks_per_stack: Number of blocks within each stack (Fixed).
        dropout_rate: Dropout rate applied within blocks (Fixed).
        activation: Activation function for hidden layers (Fixed).
        learning_rate: Optimizer learning rate (Fixed).
        loss_function: Loss function for compilation (Fixed).

    Returns:
        Compiled Keras Model.
    """

    inputs = layers.Input(shape=(input_timesteps,), name='input') 

    residual = inputs
    total_forecast = 0

    for stack_id in range(num_stacks):
        stack_forecast = 0
        for block_id in range(num_blocks_per_stack):
            block = NBeatsGenericBlock(
                input_timesteps=input_timesteps,
                output_timesteps=output_timesteps,
                layer_widths=layer_widths, 
                theta_dim=theta_dim,       
                dropout_rate=dropout_rate,
                activation=activation,
                name=f'stack_{stack_id}_block_{block_id}'
            )
            forecast, backcast = block(residual)
            residual = layers.Subtract(name=f'subtract_{stack_id}_{block_id}')([residual, backcast])
            stack_forecast = layers.Add(name=f'add_stack_forecast_{stack_id}_{block_id}')([stack_forecast, forecast])

        total_forecast = layers.Add(name=f'add_total_forecast_{stack_id}')([total_forecast, stack_forecast])

    
    model = Model(inputs=inputs, outputs=total_forecast, name='NBEATS_Generic')

    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss=loss_function, optimizer=optimizer, metrics=['mae', 'mse'])

    return model


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras



from tensorflow.keras.layers import Input, Dense, Lambda, Add, Concatenate 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


from scikeras.wrappers import KerasRegressor

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint 






def create_nbeats_model(
    
    num_stacks: int = 3,
    theta_dim: int = 128,      
    layer_widths: int = 256,   
    
    num_blocks_per_stack: int = 3,
    share_weights_in_stack: bool = False,
    stack_types: list = None, 
    
    n_timesteps: int = 50,     
    n_forecast_steps: int = 10,
    n_features: int = 1,       
    learning_rate: float = 0.001,
    loss_function: str = 'mean_squared_error',
    optimizer_name: str = 'adam'
    ):
    """
    *** PLACEHOLDER *** Creates a Keras model simulating NBEATS structure.
    Replace this with your actual NBEATS implementation compatible with Keras.
    It must accept num_stacks, theta_dim, layer_widths, and other params.
    """
    print(f"[Placeholder Model] Creating with Stacks={num_stacks}, Theta={theta_dim}, Width={layer_widths}")

    
    
    inputs = keras.Input(shape=(n_timesteps, n_features), name="Input")
    
    x = layers.Flatten()(inputs)
    
    x = layers.Dense(layer_widths, activation='relu', name=f"Hidden_Width_{layer_widths}")(x)
    x = layers.Dense(layer_widths // 2, activation='relu')(x) 
    
    x = layers.Dense(theta_dim, activation='relu', name=f"Bottleneck_Theta_{theta_dim}")(x)
    
    outputs = layers.Dense(n_forecast_steps, activation='linear', name="Output")(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name=f"Placeholder_NBEATS_S{num_stacks}")

    
    if optimizer_name.lower() == 'adam': optimizer = Adam(learning_rate=learning_rate)
    else: optimizer = Adam(learning_rate=learning_rate) 
    model.compile(loss=loss_function, optimizer=optimizer, metrics=['mae', 'mse'])
    return model



def tune_nbeats_hyperparameters(
    X_train: np.ndarray,
    y_train: np.ndarray, 
    n_iter: int = 15,
    cv: int = 3,
    
    num_stacks_range: tuple = (2, 6),      
    theta_dim_options: list = [64, 128, 256], 
    layer_width_options: list = [128, 256, 512], 
    
    n_forecast_steps: int = 10,           
    fixed_num_blocks_per_stack: int = 3,
    fixed_share_weights: bool = False,
    fixed_stack_types: list = None,       
    
    fixed_learning_rate: float = 0.001,
    fixed_batch_size: int = 64,
    fixed_epochs: int = 100,
    early_stopping_patience: int = 10,
    scoring_metric: str = 'neg_mean_squared_error', 
    random_state: int = 42
    ) -> RandomizedSearchCV:
    """
    Performs RandomizedSearchCV for an NBEATS regression model, focusing on
    num_stacks, theta_dim, and layer_widths.

    Requires a 'create_nbeats_model' function implementing the NBEATS architecture.

    Args:
        X_train: Training features (samples, n_timesteps, n_features).
        y_train: Training target values (samples, n_forecast_steps).
        n_iter: Number of parameter settings sampled by RandomizedSearchCV.
        cv: Number of cross-validation folds.
        num_stacks_range: Tuple (min, max) for sampling number of stacks.
        theta_dim_options: List of choices for the theta dimension.
        layer_width_options: List of choices for FC layer widths within blocks.
        n_forecast_steps: The number of steps the model should predict.
        fixed_num_blocks_per_stack: Fixed number of blocks per stack.
        fixed_share_weights: Fixed weight sharing setting.
        fixed_stack_types: Fixed stack types (if applicable).
        fixed_learning_rate: Learning rate to use.
        fixed_batch_size: Batch size to use.
        fixed_epochs: Max epochs for training each model.
        early_stopping_patience: Patience for EarlyStopping.
        scoring_metric: Scikit-learn scorer name.
        random_state: Seed for reproducibility.

    Returns:
        Fitted RandomizedSearchCV object containing the results, or None on error.
    """
    tf.random.set_seed(random_state)
    np.random.seed(random_state)

    
    if X_train.ndim != 3:
        raise ValueError("X_train must be 3-dimensional (samples, timesteps, features)")
    if y_train.ndim != 2 or y_train.shape[1] != n_forecast_steps:
         raise ValueError(f"y_train must be 2-dimensional (samples, n_forecast_steps={n_forecast_steps})")

    n_timesteps = X_train.shape[1]
    n_features = X_train.shape[2]

    print(f"--- Starting NBEATS Tuning ---")
    print(f"Input shape: (Timesteps={n_timesteps}, Features={n_features})")
    print(f"Output shape: (Forecast Steps={n_forecast_steps})")
    print(f"Tuning Params: num_stacks, theta_dim, layer_widths")
    print(f"Fixed Params: LR={fixed_learning_rate}, Batch={fixed_batch_size}, "
          f"Blocks={fixed_num_blocks_per_stack}, Horizon={n_forecast_steps}")
    print(f"Search: {n_iter} iterations, {cv} folds")
    print("-" * 30)

    
    param_distributions = {
        
        'model__num_stacks': randint(num_stacks_range[0], num_stacks_range[1]),
        'model__theta_dim': theta_dim_options,
        'model__layer_widths': layer_width_options,

        
        

        
        'batch_size': [fixed_batch_size] 
    }

    
    keras_regressor = KerasRegressor(
        model=create_nbeats_model, 
        
        model__n_timesteps=n_timesteps,
        model__n_forecast_steps=n_forecast_steps,
        model__n_features=n_features,
        model__num_blocks_per_stack=fixed_num_blocks_per_stack,
        model__share_weights_in_stack=fixed_share_weights,
        model__stack_types=fixed_stack_types, 
        model__learning_rate=fixed_learning_rate,
        model__loss_function='mean_squared_error', 
        model__optimizer_name='adam',             
        
        loss='mean_squared_error',
        optimizer='adam',
        metrics=['mae', 'mse'],
        
        epochs=fixed_epochs,
        random_state=random_state,
        verbose=0 
    )

    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=early_stopping_patience,
        restore_best_weights=True,
        verbose=0
    )

    
    random_search_nbeats = RandomizedSearchCV(
        estimator=keras_regressor,
        param_distributions=param_distributions,
        n_iter=n_iter,
        cv=cv,
        scoring=scoring_metric,
        verbose=2, 
        n_jobs=1, 
        random_state=random_state,
        error_score='raise'
    )

    
    try:
        print("Fitting RandomizedSearchCV...")
        
        search_result = random_search_nbeats.fit(
            X_train, y_train,
            callbacks=[early_stopping],
            validation_split=0.2 
        )
        print("\nRandomized Search Finished.")
        return search_result

    except Exception as e: 
        print(f"\nAn error occurred during Randomized Search for NBEATS.")
        print(f"Error message: {e}")
        import traceback
        traceback.print_exc() 
        return None



SEED = 42


search_results = tune_nbeats_hyperparameters(
    X_train=X_train,
    y_train=y_train,
    n_iter=5, 
    cv=2,      
    num_stacks_range=(2, 4),         
    theta_dim_options=[32, 64],     
    layer_width_options=[64, 128],  
    n_forecast_steps=y_train.shape[1], 
    fixed_num_blocks_per_stack=2,    
    fixed_learning_rate=0.005,
    fixed_batch_size=32,
    random_state=SEED
)


if search_results:
    print(f"\nBest Score ({search_results.scorer_.__name__}): {search_results.best_score_:.4f}")
    print("Best Parameters Found:")
    
    tuned_params = ['num_stacks', 'theta_dim', 'layer_widths', 'batch_size']
    best_params_display = {}
    for k, v in search_results.best_params_.items():
        clean_key = k.replace('model__', '')
        if clean_key in tuned_params or k == 'batch_size':
                best_params_display[clean_key] = v
    print(best_params_display)

    
    
    
    
    
else:
    print("\nNBEATS Hyperparameter tuning failed.")

--- Starting NBEATS Tuning ---
Input shape: (Timesteps=12, Features=15)
Output shape: (Forecast Steps=1)
Tuning Params: num_stacks, theta_dim, layer_widths
Fixed Params: LR=0.005, Batch=32, Blocks=2, Horizon=1
Search: 5 iterations, 2 folds
------------------------------
Fitting RandomizedSearchCV...
Fitting 2 folds for each of 5 candidates, totalling 10 fits
[Placeholder Model] Creating with Stacks=3, Theta=32, Width=64


2025-04-21 06:09:53.753361: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-04-21 06:09:53.911041: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}
2025-04-21 06:11:16.054463: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_un

[CV] END batch_size=32, model__layer_widths=64, model__num_stacks=3, model__theta_dim=32; total time= 1.4min
[Placeholder Model] Creating with Stacks=3, Theta=32, Width=64


2025-04-21 06:12:34.672554: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


[CV] END batch_size=32, model__layer_widths=64, model__num_stacks=3, model__theta_dim=32; total time= 1.3min
[Placeholder Model] Creating with Stacks=2, Theta=64, Width=64


...

2025-04-21 06:22:03.388313: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


[CV] END batch_size=32, model__layer_widths=64, model__num_stacks=2, model__theta_dim=64; total time= 1.4min
[Placeholder Model] Creating with Stacks=2, Theta=64, Width=64


2025-04-21 06:23:12.688023: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


[CV] END batch_size=32, model__layer_widths=64, model__num_stacks=2, model__theta_dim=64; total time= 1.2min
[Placeholder Model] Creating with Stacks=2, Theta=64, Width=64

Randomized Search Finished.


AttributeError: '_PredictScorer' object has no attribute '__name__'

In [ ]:
print(search_results.param_distributions)

RandomizedSearchCV(cv=2, error_score='raise',
                   estimator=KerasRegressor(epochs=100, loss='mean_squared_error', metrics=['mae', 'mse'], model=<function create_nbeats_model at 0x7036feb38e00>, model__learning_rate=0.005, model__loss_function='mean_squared_error', model__n_features=15, model__n_forecast_steps=1, model__n_timesteps=12, model__num_blocks_per_stack=2, mo...ack=False, model__stack_types=None, optimizer='adam', random_state=42, verbose=0),
                   n_iter=5, n_jobs=1,
                   param_distributions={'batch_size': [32],
                                        'model__layer_widths': [64, 128],
                                        'model__num_stacks': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7036ff2f67e0>,
                                        'model__theta_dim': [32, 64]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)


In [16]:
print(search_results.best_params_)

{'batch_size': 32, 'model__layer_widths': 64, 'model__num_stacks': 2, 'model__theta_dim': 64}


In [15]:
print(search_results.param_distributions["model__num_stacks"])

In [ ]:
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform 

def tune_nbeats_hyperparameters(
    X_train: np.ndarray,
    y_train: np.ndarray, 
    output_timesteps: int,
    n_iter: int = 5,
    cv: int = 2,
    
    num_stacks_range: tuple = (2, 6),      
    theta_dim_options: list = [64, 128, 256], 
    layer_width_options: list = [128, 256, 512], 
    
    fixed_num_blocks_per_stack: int = 3,
    fixed_learning_rate: list = [0.001,0.002,0.003], 
    fixed_batch_size: int = 128,
    fixed_dropout_rate: float = 0.1,
    fixed_activation: str = 'relu',
    fixed_epochs: int = 100,
    early_stopping_patience: int = 10,
    scoring_metric: str = 'neg_mean_squared_error', 
    random_state: int = 42
    ) -> RandomizedSearchCV:
    """
    Performs RandomizedSearchCV for a Generic N-BEATS regression model, focusing on
    num_stacks, theta_dim (as bottleneck), and layer_widths.

    Args:
        X_train: Training features (samples, input_timesteps). N-BEATS often takes flat input.
        y_train: Training target values (samples, output_timesteps).
        output_timesteps: The forecast horizon length.
        n_iter: Number of parameter settings sampled by RandomizedSearchCV.
        cv: Number of cross-validation folds.
        num_stacks_range: Tuple (min, max) for randint sampling of stack count.
        theta_dim_options: List of choices for the block's bottleneck dimension.
        layer_width_options: List of choices for the hidden layer width within blocks.
        fixed_num_blocks_per_stack: Fixed number of blocks per stack.
        fixed_learning_rate: Learning rate to use (fixed).
        fixed_batch_size: Batch size to use (fixed).
        fixed_dropout_rate: Dropout rate to use.
        fixed_activation: Activation function ('relu' or 'gelu').
        fixed_epochs: Max epochs for training each model.
        early_stopping_patience: Patience for EarlyStopping callback.
        scoring_metric: Scikit-learn scorer name for evaluation.
        random_state: Seed for reproducibility.

    Returns:
        Fitted RandomizedSearchCV object containing the results.
    """
    tf.random.set_seed(random_state)
    np.random.seed(random_state)

    
    if X_train.ndim != 2:
        raise ValueError(f"X_train must be 2-dimensional (samples, input_timesteps) for this N-BEATS setup. Found {X_train.ndim} dims.")
    input_timesteps = X_train.shape[1]

    
    if y_train.ndim != 2 or y_train.shape[1] != output_timesteps:
         raise ValueError(f"y_train must be 2-dimensional (samples, output_timesteps={output_timesteps}). Found shape {y_train.shape}.")


    print(f"--- Starting N-BEATS Tuning ---")
    print(f"Input shape: (Timesteps={input_timesteps})")
    print(f"Output shape: (Timesteps={output_timesteps})")
    print(f"Tuning Params: num_stacks, theta_dim, layer_widths (single hidden layer width)")
    print(f"Fixed Params: LR={fixed_learning_rate}, Batch={fixed_batch_size}, "
          f"Dropout={fixed_dropout_rate}, Blocks/Stack={fixed_num_blocks_per_stack}, Act={fixed_activation}")
    print(f"Search: {n_iter} iterations, {cv} folds")
    print("-" * 30)

    
    param_distributions = {
        
        'model__num_stacks': randint(num_stacks_range[0], num_stacks_range[1]),
        'model__theta_dim': theta_dim_options,
        
        
        
        'layer_width_single': layer_width_options,

        
        'batch_size': [fixed_batch_size] 
    }

    
    
    keras_regressor = KerasRegressor(
        model=create_nbeats_model,
        
        model__input_timesteps=input_timesteps,
        model__output_timesteps=output_timesteps,
        model__num_blocks_per_stack=fixed_num_blocks_per_stack,
        model__dropout_rate=fixed_dropout_rate,
        model__activation=fixed_activation,
        model__learning_rate=fixed_learning_rate,
        model__loss_function='mean_squared_error',
        
        loss='mean_squared_error',
        optimizer='adam',
        metrics=['mae', 'mse'],
        
        epochs=fixed_epochs,
        
        
        
        
        
        
        
        
        
        
        
        
        

        random_state=random_state,
        verbose=0 
    )

    
    def model_wrapper_nbeats(**kwargs):
        
        single_width = kwargs.pop('layer_width_single', 256) 
        
        kwargs['layer_widths'] = [single_width, single_width] 
        
        return create_nbeats_model(**kwargs)

    
    keras_regressor.set_params(model=model_wrapper_nbeats)

    
    param_distributions = {
        'model__num_stacks': randint(num_stacks_range[0], num_stacks_range[1]),
        'model__theta_dim': theta_dim_options,
        'model__layer_width_single': layer_width_options, 
        'batch_size': [fixed_batch_size]
    }


    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=early_stopping_patience,
        restore_best_weights=True,
        verbose=0
    )

    
    random_search_nbeats = RandomizedSearchCV(
        estimator=keras_regressor,
        param_distributions=param_distributions,
        n_iter=n_iter,
        cv=cv,
        scoring=scoring_metric,
        verbose=2,
        n_jobs=1,
        random_state=random_state,
        error_score='raise'
    )

    
    try:
        print("Fitting RandomizedSearchCV...")
        
        search_result = random_search_nbeats.fit(
            X_train, y_train, 
            callbacks=[early_stopping],
            validation_split=0.2 
        )
        print("\nRandomized Search Finished.")
        return search_result

    except Exception as e:
        print(f"\nAn error occurred during Randomized Search for N-BEATS.")
        print(f"Error message: {e}")
        import traceback
        traceback.print_exc() 
        return None 






print(f"Demo Data Shapes: X={X_train.shape}, y={y_train.shape}")



SEED = 42 
search_results_nbeats = tune_nbeats_hyperparameters(
    X_train=X_train,
    y_train=y_train,
    output_timesteps=12,
    n_iter=5, 
    cv=2,     
    num_stacks_range=(2, 4),         
    theta_dim_options=[64, 128],     
    layer_width_options=[128, 256],  
    fixed_learning_rate=0.002,
    fixed_batch_size=64,
    random_state=SEED
)


if search_results_nbeats:
    print(f"\nBest Score ({search_results_nbeats.scorer_.__name__}): {search_results_nbeats.best_score_:.4f}")
    print("Best Parameters Found:")
    best_params_display = {k.replace('model__', ''): v for k, v in search_results_nbeats.best_params_.items()}
    print(best_params_display)

    
    
    
    
    
else:
    print("\nN-BEATS Hyperparameter tuning failed.")


Demo Data Shapes: X=(26262, 12, 15), y=(26262, 1)


ValueError: X_train must be 2-dimensional (samples, input_timesteps) for this N-BEATS setup. Found 3 dims.